In [62]:
import numpy as np
import pandas as pd
import keras
from keras import regularizers, optimizers
from keras.layers import Input, Conv1D, Dense, Flatten, Activation, UpSampling1D, MaxPooling1D, ZeroPadding1D
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.models import Model, load_model
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing.data import StandardScaler

In [63]:
df = pd.read_csv("datasets/creditcardfraud/creditcard.csv", sep=",", index_col=None)
print(df.shape)
df.head()

(284807, 31)


Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V27       V28  Amount  Class  
0 -0.189115  0.133558 -0.021053  149.62      0  
1  0.125895 -0.008983  0.014724    2.69      0  
2 -0.139097 -0.055353 -0.059752  378.66      0  
3 -0.221929  0.062723  0.061458  123.50      0  
4  0.502292  0.219422  0.215153   69.99      0  

[5 rows x 31 columns]

In [64]:
df.tail()

Time         V1         V2        V3        V4        V5  \
284802  172786.0 -11.881118  10.071785 -9.834783 -2.066656 -5.364473   
284803  172787.0  -0.732789  -0.055080  2.035030 -0.738589  0.868229   
284804  172788.0   1.919565  -0.301254 -3.249640 -0.557828  2.630515   
284805  172788.0  -0.240440   0.530483  0.702510  0.689799 -0.377961   
284806  172792.0  -0.533413  -0.189733  0.703337 -0.506271 -0.012546   

              V6        V7        V8        V9  ...       V21       V22  \
284802 -2.606837 -4.918215  7.305334  1.914428  ...  0.213454  0.111864   
284803  1.058415  0.024330  0.294869  0.584800  ...  0.214205  0.924384   
284804  3.031260 -0.296827  0.708417  0.432454  ...  0.232045  0.578229   
284805  0.623708 -0.686180  0.679145  0.392087  ...  0.265245  0.800049   
284806 -0.649617  1.577006 -0.414650  0.486180  ...  0.261057  0.643078   

             V23       V24       V25       V26       V27       V28  Amount  \
284802  1.014480 -0.509348  1.436807  0.250034  0.943651  0.823731    0.77   
284803  0.012463 -1.016226 -0.606624 -0.395255  0.068472 -0.053527   24.79   
284804 -0.037501  0.640134  0.265745 -0.087371  0.004455 -0.026561   67.88   
284805 -0.163298  0.123205 -0.569159  0.546668  0.108821  0.104533   10.00   
284806  0.376777  0.008797 -0.473649 -0.818267 -0.002415  0.013649  217.00   

        Class  
284802      0  
284803      0  
284804      0  
284805      0  
284806      0  

[5 rows x 31 columns]

In [65]:
df['Amount'] = StandardScaler().fit_transform(df['Amount'].values.reshape(-1, 1))
df['Time'] = StandardScaler().fit_transform(df['Time'].values.reshape(-1, 1))
df.tail()

Time         V1         V2        V3        V4        V5  \
284802  1.641931 -11.881118  10.071785 -9.834783 -2.066656 -5.364473   
284803  1.641952  -0.732789  -0.055080  2.035030 -0.738589  0.868229   
284804  1.641974   1.919565  -0.301254 -3.249640 -0.557828  2.630515   
284805  1.641974  -0.240440   0.530483  0.702510  0.689799 -0.377961   
284806  1.642058  -0.533413  -0.189733  0.703337 -0.506271 -0.012546   

              V6        V7        V8        V9  ...       V21       V22  \
284802 -2.606837 -4.918215  7.305334  1.914428  ...  0.213454  0.111864   
284803  1.058415  0.024330  0.294869  0.584800  ...  0.214205  0.924384   
284804  3.031260 -0.296827  0.708417  0.432454  ...  0.232045  0.578229   
284805  0.623708 -0.686180  0.679145  0.392087  ...  0.265245  0.800049   
284806 -0.649617  1.577006 -0.414650  0.486180  ...  0.261057  0.643078   

             V23       V24       V25       V26       V27       V28    Amount  \
284802  1.014480 -0.509348  1.436807  0.250034  0.943651  0.823731 -0.350151   
284803  0.012463 -1.016226 -0.606624 -0.395255  0.068472 -0.053527 -0.254117   
284804 -0.037501  0.640134  0.265745 -0.087371  0.004455 -0.026561 -0.081839   
284805 -0.163298  0.123205 -0.569159  0.546668  0.108821  0.104533 -0.313249   
284806  0.376777  0.008797 -0.473649 -0.818267 -0.002415  0.013649  0.514355   

        Class  
284802      0  
284803      0  
284804      0  
284805      0  
284806      0  

[5 rows x 31 columns]

In [66]:
anomalies = df[df["Class"] == 1]
normal = df[df["Class"] == 0]

anomalies.shape, normal.shape


((492, 31), (284315, 31))

In [67]:

for f in range(0, 20):
    normal = normal.iloc[np.random.permutation(len(normal))]
    

data_set = pd.concat([normal[:10000], anomalies])

x_train, x_test = train_test_split(data_set, test_size = 0.4, random_state = 42)

x_train = x_train.sort_values(by=['Time'])
x_test = x_test.sort_values(by=['Time'])

y_train = x_train["Class"]
y_test = x_test["Class"]

x_train.head(10)

Time        V1        V2        V3        V4        V5        V6  \
21  -1.996225  0.962496  0.328461 -0.171479  2.109204  1.129566  1.696038   
115 -1.995025  1.038370  0.127486  0.184456  1.109950  0.441699  0.945283   
182 -1.994098  1.254914  0.350287  0.302488  0.693114 -0.371470 -1.070256   
222 -1.993530 -2.419486  1.949346  0.552998  0.982710 -0.284815  2.411200   
464 -1.989402 -0.268621 -0.233374  1.025263 -2.129325  1.566375  3.863064   
559 -1.987802 -2.680348  1.872052  1.144712 -0.693664  0.155172  0.601325   
602 -1.987002  1.179162  0.205327  0.704971  0.697812 -0.566141 -0.830944   
604 -1.986960 -0.942973  1.812575  2.112754  2.211791 -0.067565 -0.614476   
623 -1.986644 -3.043541 -3.157307  1.088463  2.288644  1.359805 -1.064823   
651 -1.986223 -0.789890 -1.379371  0.171334 -1.636756 -2.807266  0.726236   

           V7        V8        V9  ...       V21       V22       V23  \
21   0.107712  0.521502 -1.191311  ...  0.143997  0.402492 -0.048508   
115 -0.036715  0.350995  0.118950  ...  0.102520  0.605089  0.023092   
182  0.086781 -0.202836  0.035154  ... -0.287592 -0.832682  0.128083   
222 -1.398537 -0.188922  0.675695  ...  1.213390 -1.238354  0.007191   
464 -0.732265  0.807716 -1.230053  ...  0.343692  0.921648 -0.410791   
559  0.904201 -0.520079  3.013065  ... -0.459592  0.485421 -0.365437   
602 -0.013824 -0.093148  0.026189  ... -0.196312 -0.543615  0.233359   
604  0.775533 -0.078299 -1.276690  ... -0.181812 -0.441114  0.004498   
623  0.325574 -0.067794 -0.270953  ...  0.661696  0.435477  1.375966   
651  2.737602 -0.933999 -2.413730  ... -0.256231  0.167077  0.540876   

          V24       V25       V26       V27       V28    Amount  Class  
21  -1.371866  0.390814  0.199964  0.016371 -0.014605 -0.216935      0  
115 -0.626463  0.479120 -0.166937  0.081247  0.001192 -0.348512      0  
182  0.339427  0.215944  0.094704 -0.023354  0.030892 -0.342475      0  
222 -1.724175  0.239721 -0.313607 -0.187431  0.119472 -0.326282      0  
464  1.052995  0.398555  0.070685 -0.070437 -0.148873 -0.265271      0  
559 -0.744118  0.328655  0.457695  0.566152  0.168241 -0.233327      0  
602  0.597346  0.068706  0.091725 -0.012834  0.020738 -0.345313      0  
604  0.881532 -0.169525 -0.213305  0.313371  0.246707 -0.342355      0  
623 -0.293803  0.279798 -0.145362 -0.252773  0.035764  1.761758      1  
651  0.067496  0.479636 -0.098230  0.053527 -0.408050  2.175160      0  

[10 rows x 31 columns]

In [68]:
print("Shapes:\nx_train:%s\ny_train:%s\n" % (x_train.shape, y_train.shape))
print("x_test:%s\ny_test:%s\n" % (x_test.shape, y_test.shape))


Shapes:
x_train:(6295, 31)
y_train:(6295,)

x_test:(4197, 31)
y_test:(4197,)



In [69]:
x_train = np.array(x_train).reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = np.array(x_test).reshape(x_test.shape[0], x_test.shape[1], 1)
input_shape = (x_train.shape[1], 1)

y_train = keras.utils.to_categorical(y_train, 2)
y_test = keras.utils.to_categorical(y_test, 2)


In [70]:
print("Shapes:\nx_train:%s\ny_train:%s\n" % (x_train.shape, y_train.shape))
print("x_test:%s\ny_test:%s\n" % (x_test.shape, y_test.shape))
print("input_shape:{}\n".format(input_shape))

Shapes:
x_train:(6295, 31, 1)
y_train:(6295, 2)

x_test:(4197, 31, 1)
y_test:(4197, 2)

input_shape:(31, 1)



In [71]:

input_layer = Input(shape=(input_shape ))

### ENCODING STAGE
# Pairs of causal 1D convolutional layers and pooling layers comprising the encoding stage
conv_1 = Conv1D(filters=int(input_shape[0]), kernel_size=2, dilation_rate=1,
                padding='causal', strides=1,input_shape=input_shape,
                kernel_regularizer=regularizers.l2(0.01),
                activation='relu')(input_layer)

pool_1 = MaxPooling1D(pool_size=2, strides=2)(conv_1)

conv_2 = Conv1D(filters=int(input_shape[0] / 2), kernel_size=2, dilation_rate=1,
                padding='causal',strides=1, kernel_regularizer=regularizers.l2(0.01),
                activation='relu')(pool_1)

pool_2 = MaxPooling1D(pool_size=2, strides=3)(conv_2)

conv_3 = Conv1D(filters=int(input_shape[0] / 3), kernel_size=2, dilation_rate=1,
                padding='causal', strides=1,kernel_regularizer=regularizers.l2(0.01),
                activation='relu')(pool_2)


### OUTPUT OF ENCODING STAGE
encoder = Dense(int(input_shape[0] / 6), activation='relu')(conv_3)

### DECODING STAGE
# Pairs of upsampling and causal 1D convolutional layers comprising the decoding stage
upsample_1 = UpSampling1D(size=3)(encoder)

conv_4 = Conv1D(filters=int(input_shape[0]/3), kernel_size=2, dilation_rate=1,
                padding='causal',strides=1, kernel_regularizer=regularizers.l2(0.01),
                activation='relu')(upsample_1)

upsample_2 = UpSampling1D(size=2)(conv_4)

conv_5 = Conv1D(filters=int(input_shape[0]/2), kernel_size=2, dilation_rate=1,
                padding='causal', strides=1,kernel_regularizer=regularizers.l2(0.05),
                activation='relu')(upsample_2)

zero_pad_1 = ZeroPadding1D(padding=(0,1))(conv_5)

conv_6 = Conv1D(filters=int(input_shape[0]), kernel_size=2, dilation_rate=1,
                padding='causal', strides=1,kernel_regularizer=regularizers.l2(0.05),
                activation='relu')(zero_pad_1)


### Output of decoding stage flattened and passed through softmax to make predictions
flat = Flatten()(conv_6)

output_layer = Dense(2, activation='softmax')(flat)

TCN = Model(inputs=input_layer, outputs=output_layer)

In [72]:
TCN.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=optimizers.Adam(lr=0.002),
           metrics=["accuracy"])

checkpointer = ModelCheckpoint(filepath="model_ED-TCN_creditcard.h5",
                               verbose=0,
                               save_best_only=True)

TCN.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        (None, 31, 1)             0         
_________________________________________________________________
conv1d_101 (Conv1D)          (None, 31, 31)            93        
_________________________________________________________________
max_pooling1d_35 (MaxPooling (None, 15, 31)            0         
_________________________________________________________________
conv1d_102 (Conv1D)          (None, 15, 15)            945       
_________________________________________________________________
max_pooling1d_36 (MaxPooling (None, 5, 15)             0         
_________________________________________________________________
conv1d_103 (Conv1D)          (None, 5, 10)             310       
_________________________________________________________________
dense_32 (Dense)             (None, 5, 5)              55        
__________

In [73]:
TCN.fit(x_train, y_train,
          batch_size=128,
          epochs=25,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks = [checkpointer])


Train on 6295 samples, validate on 4197 samples
Epoch 1/25
6295/6295 [==============================] - 4s - loss: 1.6630 - acc: 0.9360 - val_loss: 0.8879 - val_acc: 0.9523
Epoch 2/25
6295/6295 [==============================] - 1s - loss: 0.5690 - acc: 0.9536 - val_loss: 0.3316 - val_acc: 0.9523
Epoch 3/25
6295/6295 [==============================] - 1s - loss: 0.2272 - acc: 0.9844 - val_loss: 0.1581 - val_acc: 0.9893
Epoch 4/25
6295/6295 [==============================] - 0s - loss: 0.1278 - acc: 0.9889 - val_loss: 0.1128 - val_acc: 0.9881
Epoch 5/25
6295/6295 [==============================] - 0s - loss: 0.1004 - acc: 0.9890 - val_loss: 0.0954 - val_acc: 0.9898
Epoch 6/25
6295/6295 [==============================] - 0s - loss: 0.0925 - acc: 0.9897 - val_loss: 0.1023 - val_acc: 0.9893
Epoch 7/25
6295/6295 [==============================] - 0s - loss: 0.0891 - acc: 0.9889 - val_loss: 0.0909 - val_acc: 0.9907
Epoch 8/25
6295/6295 [==============================] - 0s - loss: 0.0848 - a

In [74]:

score = TCN.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.0717651191317761
Test accuracy: 0.9904693828925423


In [75]:
from sklearn.metrics import roc_auc_score

preds = TCN.predict(x_test)
auc = roc_auc_score( np.round(preds), y_test)
print("AUC: {:.2%}".format (auc))

AUC: 98.64%


In [13]:

predictions = TCN.predict(x_test)

mse = np.mean(np.power(x_test - predictions, 2), axis=1)

In [20]:
predictions[0].shape

(30, 1)

In [16]:
for f in range(0, len(x_test)):
    print(x_test[f], predictions[f])

[[-1.99605658]
 [ 1.1140086 ]
 [ 0.08554609]
 [ 0.49370249]
 [ 1.33575999]
 [-0.30018855]
 [-0.01075378]
 [-0.11876002]
 [ 0.1886167 ]
 [ 0.20568685]
 [ 0.08226226]
 [ 1.13355567]
 [ 0.626699  ]
 [-1.49278039]
 [ 0.52078789]
 [-0.6745926 ]
 [-0.52910824]
 [ 0.1582562 ]
 [-0.39875148]
 [-0.14570891]
 [-0.27383237]
 [-0.05323366]
 [-0.00476015]
 [-0.03147017]
 [ 0.19805372]
 [ 0.56500731]
 [-0.33771813]
 [ 0.0290574 ]
 [ 0.00445263]
 [-0.33543791]] [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
[[-1.99603552e+00]
 [-5.29912284e-01]
 [ 8.73891581e-01]
 [ 1.34724733e+00]
 [ 1.45456677e-01]
 [ 4.14208858e-01]
 [ 1.00223094e-01]
 [ 7.11206083e-01]
 [ 1.76065957e-01]
 [-2.86716935e-01]
 [-4.84687683e-01]
 [ 8.72489590e-01]
 [ 8.51635860e-01]
 [-5.71745303e-01]
 [ 1.00974273e-01]
 [-1.51977183e+00]
 [-2.84375978e-01]
 [-3.10523585e-01]
 [-4.04247869e-01]
 [-8.23

 [-0.3496711 ]] [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
[[-1.65285453]
 [-1.45293124]
 [-1.19553896]
 [ 2.0789652 ]
 [ 0.78831321]
 [-1.36913976]
 [ 1.26724142]
 [ 1.62862398]
 [-0.8991001 ]
 [ 1.83709291]
 [ 0.01860847]
 [ 2.13619002]
 [-2.09171488]
 [ 2.09210886]
 [ 0.36587268]
 [-0.1300233 ]
 [-0.35926207]
 [ 0.49609165]
 [ 0.74189703]
 [ 2.30080839]
 [ 0.05451938]
 [-0.45157666]
 [ 0.08950795]
 [-0.19022009]
 [ 0.11361994]
 [-0.53775396]
 [ 1.09530897]
 [-0.39342826]
 [-0.49098781]
 [ 1.6458137 ]] [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
[[-1.64683197]
 [-1.16034691]
 [ 0.27360458]
 [ 2.2465346 ]
 [ 0.40061343]
 [ 0.67046123]
 [-0.13695582]
 [ 0.13220413]
 [ 0.18103947]
 [ 0.71193398]
 [-0.98215207]
 [ 2.2

 [1.]]
[[-1.38979895]
 [ 1.41377505]
 [-0.76383805]
 [ 1.01535048]
 [-0.81894044]
 [-1.34435063]
 [-0.04752102]
 [-1.32416213]
 [ 0.07563981]
 [-0.15743828]
 [ 0.57680493]
 [-1.04337266]
 [-0.75107234]
 [ 0.82354797]
 [-0.73108658]
 [ 1.34498972]
 [ 1.93402805]
 [-0.37012802]
 [-0.39842883]
 [ 0.41596501]
 [ 0.14163864]
 [ 0.38261286]
 [ 1.09049652]
 [-0.18765694]
 [-0.4044479 ]
 [ 0.43620901]
 [ 0.00891699]
 [ 0.05888973]
 [ 0.02500904]
 [-0.31328851]] [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
[[-1.38977789]
 [ 1.25390039]
 [-0.08737129]
 [-0.76360872]
 [-0.04269786]
 [ 1.85315687]
 [ 3.5998954 ]
 [-0.80654228]
 [ 0.91737253]
 [ 0.18434061]
 [ 0.03708952]
 [-0.19755766]
 [ 0.03628032]
 [ 0.10808502]
 [ 0.26184299]
 [ 1.42538383]
 [ 0.71711038]
 [-1.09312257]
 [ 0.64540065]
 [-0.20003308]
 [ 0.00903509]
 [ 0.08195194]
 [ 0.1329199 ]
 [-0.12273929]
 

[[-1.26385151]
 [-0.84194623]
 [ 1.1429455 ]
 [ 1.79008545]
 [ 2.48253574]
 [ 0.79008182]
 [ 2.99626758]
 [-0.017301  ]
 [ 0.55561308]
 [-0.18224124]
 [ 1.26125377]
 [-0.6317122 ]
 [-0.17698637]
 [ 0.28356547]
 [-0.66049389]
 [ 1.46813194]
 [-1.45501355]
 [ 1.02113218]
 [-1.51931068]
 [-0.36895716]
 [ 0.29361984]
 [ 0.14629839]
 [ 1.10142516]
 [-0.19873198]
 [-1.76576134]
 [-0.48580264]
 [ 0.47767342]
 [ 0.19464667]
 [ 0.03144303]
 [-0.23824443]] [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
[[-1.26353564e+00]
 [-6.03670496e-01]
 [ 1.05508131e+00]
 [ 7.57501212e-01]
 [-1.25712405e-01]
 [-1.06639236e-01]
 [-5.64457467e-01]
 [ 2.70281293e-01]
 [ 4.83020267e-01]
 [-7.37319622e-01]
 [-4.45082890e-01]
 [ 4.81582376e-01]
 [-1.42319073e-01]
 [-1.39164877e+00]
 [ 1.02687091e+00]
 [ 8.77840522e-02]
 [ 4.81954321e-01]
 [-4.77630782e-01]
 [ 6.69212440e-01]
 [ 3.77

 [1.]]
[[-1.16348944]
 [-2.86909239]
 [ 3.52678788]
 [-3.12392739]
 [ 0.3697753 ]
 [-0.78795707]
 [-0.22304554]
 [-1.29540345]
 [ 2.57259211]
 [-0.82977723]
 [ 0.55934835]
 [-1.0368183 ]
 [ 1.45992013]
 [ 1.2505203 ]
 [ 1.99353476]
 [-0.14730968]
 [ 0.73402826]
 [-0.09724767]
 [ 0.99951378]
 [ 1.13853909]
 [ 0.14369121]
 [ 0.18574973]
 [ 0.28519578]
 [ 0.16136101]
 [-1.38974726]
 [-0.08696193]
 [-0.29081206]
 [ 0.21239667]
 [ 0.1300975 ]
 [-0.33887626]] [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
[[-1.16315252]
 [-0.75284457]
 [ 1.04590119]
 [ 0.89010695]
 [ 0.93537181]
 [ 0.14018581]
 [-0.43349438]
 [ 0.32301192]
 [ 0.39353622]
 [-0.85827712]
 [-0.49518166]
 [-0.42501468]
 [ 0.23690162]
 [ 0.28280717]
 [ 0.66186216]
 [ 1.21111735]
 [-0.5025185 ]
 [ 0.24879027]
 [-0.31360653]
 [ 0.06349112]
 [-0.04675731]
 [ 0.25548019]
 [ 0.60905592]
 [-0.06380158]
 

...

 [-0.09143471]] [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
[[ 1.04079084]
 [ 1.9571696 ]
 [-0.37891874]
 [-1.5353829 ]
 [ 0.92151729]
 [-0.2665261 ]
 [-0.44563401]
 [-0.57368362]
 [ 0.00632443]
 [-0.20369219]
 [ 0.11128201]
 [-0.75477042]
 [-0.98038629]
 [-0.846364  ]
 [-2.04558872]
 [ 0.56700858]
 [-0.61816697]
 [ 1.42947119]
 [ 2.36615968]
 [-1.63340023]
 [-0.53031836]
 [-0.41833468]
 [-0.75415882]
 [ 0.22751016]
 [ 0.38647934]
 [-0.21268287]
 [-0.70062993]
 [ 0.07468082]
 [ 0.03148602]
 [-0.09335379]] [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
[[ 1.04156998]
 [ 1.83096188]
 [-0.72177635]
 [-0.78981841]
 [ 0.09354063]
 [ 0.50873207]
 [ 2.31967812]
 [-1.10001735]
 [ 0.77748266]
 [ 1.10454466]
 [ 0.04130186]
 [ 0.6

 [-0.34607282]] [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
[[ 1.14014212]
 [ 2.0058379 ]
 [ 0.03599728]
 [-1.98195077]
 [ 0.47516469]
 [ 0.4683173 ]
 [-1.49321198]
 [ 0.87091198]
 [-0.59635674]
 [ 0.24292017]
 [-0.05523553]
 [-0.65630355]
 [ 0.81268803]
 [ 0.6085107 ]
 [ 0.51278336]
 [-0.24485058]
 [-0.98458256]
 [-0.08914514]
 [-0.75271609]
 [ 0.06758558]
 [-0.13489623]
 [ 0.17516548]
 [ 0.68727462]
 [-0.12085573]
 [ 0.06193733]
 [ 0.55558977]
 [-0.07295627]
 [-0.05295954]
 [-0.06516901]
 [-0.15332508]] [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
[[ 1.1406054 ]
 [-1.79026359]
 [-0.78510338]
 [-0.22609204]
 [-2.4130234 ]
 [ 0.62262743]
 [ 0.80326239]
 [ 0.76290342]
 [ 0.68663746]
 [-2.01468116]
 [-0.65139627]
 [ 0.5

 [-0.01579092]] [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
[[ 1.25535002]
 [-1.62128388]
 [ 0.2699038 ]
 [-0.53928397]
 [ 0.16608055]
 [ 1.04728765]
 [-0.44572101]
 [-0.49252318]
 [ 0.72515684]
 [-0.36538439]
 [-0.8570185 ]
 [-1.6221883 ]
 [ 0.84349271]
 [ 1.58409914]
 [ 0.55133777]
 [ 0.55045209]
 [-0.2994486 ]
 [-0.01266298]
 [ 0.11004925]
 [ 1.1901634 ]
 [-0.18991046]
 [ 0.16782581]
 [ 0.33882902]
 [-0.50684612]
 [ 0.1237634 ]
 [-0.41714402]
 [-0.24909427]
 [ 0.04724026]
 [-0.2441844 ]
 [-0.32880109]] [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
[[ 1.25625551]
 [ 2.0738624 ]
 [ 0.18296969]
 [-1.68247116]
 [ 0.42194524]
 [ 0.42714064]
 [-0.89280473]
 [ 0.20633843]
 [-0.25587479]
 [ 0.44564383]
 [-0.38695522]
 [-0.4

 [1.]]
[[ 1.38199238]
 [ 2.09594822]
 [-0.82340981]
 [-1.55248202]
 [-0.76928384]
 [-0.10356232]
 [-0.11597701]
 [-0.41154597]
 [-0.11996017]
 [-0.83981471]
 [ 1.09216153]
 [ 0.26792108]
 [ 0.47981181]
 [ 0.73543734]
 [ 0.30929353]
 [-0.07587803]
 [-0.95115929]
 [-0.77844307]
 [ 1.7522368 ]
 [-0.46611838]
 [-0.42039339]
 [-0.19289219]
 [-0.15621518]
 [ 0.10166395]
 [ 0.1151346 ]
 [-0.129079  ]
 [ 0.66182065]
 [-0.07311243]
 [-0.0602557 ]
 [-0.12381921]] [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
[[ 1.38348749]
 [ 2.0885952 ]
 [-0.87407548]
 [-1.25347704]
 [-0.80801837]
 [-0.70773878]
 [-1.3378222 ]
 [-0.21293069]
 [-0.39042867]
 [-0.46240672]
 [ 0.78163952]
 [-0.67670009]
 [-0.5851975 ]
 [-0.06144147]
 [ 0.05211047]
 [ 0.10412809]
 [ 0.91591485]
 [ 0.23832744]
 [-1.78233237]
 [ 0.75187704]
 [ 0.07797711]
 [-0.02533025]
 [-0.25785507]
 [ 0.23443789]
 

 [1.]]
[[ 1.47923787]
 [ 0.02258733]
 [ 0.79831682]
 [ 0.1472458 ]
 [-0.80278847]
 [ 0.69279693]
 [-0.49053293]
 [ 0.88006729]
 [-0.00751827]
 [-0.27483476]
 [-0.24814457]
 [ 0.4624285 ]
 [ 0.70501892]
 [-0.02744676]
 [ 0.29422248]
 [-1.02523693]
 [ 0.1615138 ]
 [-0.74681593]
 [-0.2100848 ]
 [ 0.28455472]
 [-0.02512004]
 [-0.23018451]
 [-0.51019084]
 [ 0.01167965]
 [-0.47334048]
 [-0.48237238]
 [ 0.15080186]
 [ 0.246355  ]
 [ 0.0824991 ]
 [-0.34607282]] [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
[[ 1.47936421]
 [ 1.8698821 ]
 [-0.62719866]
 [-0.30821114]
 [ 0.57513756]
 [-0.90256796]
 [-0.52950058]
 [-0.59619857]
 [-0.01923452]
 [ 1.56411927]
 [-0.17440596]
 [-1.05627592]
 [ 0.16699062]
 [-0.49290153]
 [-0.06595678]
 [ 0.59449085]
 [ 0.03779371]
 [-0.40498263]
 [ 0.2498204 ]
 [-0.34670449]
 [-0.16568942]
 [ 0.27838379]
 [ 0.93634254]
 [ 0.05257895]
 

In [56]:

predictions = TCN.predict(x_test)

mse = np.mean(np.power(x_test - predictions, 2), axis=1)
error_df = pd.DataFrame({'reconstruction_error': mse,
                        'true_class': y_test}, index=[0])

error_df.describe()

Exception: Data must be 1-dimensional